### Audio to MIDI

In [1]:
from basic_pitch import ICASSP_2022_MODEL_PATH, inference

scikit-learn version 1.5.1 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.


TensorFlow version 2.15.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Failed to load _MLModelProxy: No module named 'coremltools.libcoremlpython'
Fail to import BlobReader from libmilstoragepython. No module named 'coremltools.libmilstoragepython'
Fail to import BlobWriter from libmilstoragepython. No module named 'coremltools.libmilstoragepython'


In [2]:
def mp3_to_midi(audio_path, midi_path):
    _, midi_data, __ = inference.predict(
        audio_path,    
        model_or_model_path = ICASSP_2022_MODEL_PATH, 
        onset_threshold = 0.6, #note segmentation 1) how easily a note should be split into two. (split notes <- ..0.5.. -> merge notes)
        frame_threshold = 0.6, #model confidence threshold 2) the model confidence required to create a note. (more notes <- ..0.3.. -> fewer notes)
    )

    for instrument in midi_data.instruments:
        instrument.program = 30 #distortion guitar program

    midi_data.write(midi_path)

In [3]:
audio_path = "input_audio_to_midi/3.mp3"
midi_path = "output_audio_to_midi/3.mid"

mp3_to_midi(audio_path, midi_path)

Predicting MIDI for input_audio_to_midi/3.mp3...


Other method to create the midi file

In [ ]:
#inference.predict_and_save(
#    audio_path_list = [audio_path],
#    output_directory = "output_audio_to_midi",
#    save_midi = True,
#    sonify_midi = False,
#    save_model_outputs = False,
#    save_notes = False,
#    model_or_model_path = ICASSP_2022_MODEL_PATH,
#    #midi_tempo = tempo, #BPM
#    onset_threshold = 0.6, #note segmentation 1)
#    frame_threshold = 0.6, #model confidence threshold 2)
#    multiple_pitch_bends = True
#)

#### Play back the midi file 

In [98]:
import tinysoundfont
import time

synth = tinysoundfont.Synth()
sfid = synth.sfload("FluidR3_GM.sf2")

seq = tinysoundfont.Sequencer(synth)
seq.midi_load("output_audio_to_midi/scala_basic_pitch.mid")

# Larger buffer because latency is not important
synth.start(buffer_size=4096)

while not seq.is_empty():
    time.sleep(0.5)

#### Inspecting the .sfx file

In [2]:
import tinysoundfont

# Load the SoundFont
synth = tinysoundfont.Synth()
sfid = synth.sfload("FluidR3_GM.sf2")

# List all the presets in the SoundFont
for bank in range(128):  # MIDI supports up to 128 banks
    for preset in range(128):  # Each bank has up to 128 presets
        preset_name = synth.sfpreset_name(sfid, bank, preset)
        if preset_name:
            print(f"Bank {bank}, Preset {preset}: {preset_name}")

Bank 0, Preset 0: Yamaha Grand Piano
Bank 0, Preset 1: Bright Yamaha Grand
Bank 0, Preset 2: Electric Piano
Bank 0, Preset 3: Honky Tonk
Bank 0, Preset 4: Rhodes EP
Bank 0, Preset 5: Legend EP 2
Bank 0, Preset 6: Harpsichord
Bank 0, Preset 7: Clavinet
Bank 0, Preset 8: Celesta
Bank 0, Preset 9: Glockenspiel
Bank 0, Preset 10: Music Box
Bank 0, Preset 11: Vibraphone
Bank 0, Preset 12: Marimba
Bank 0, Preset 13: Xylophone
Bank 0, Preset 14: Tubular Bells
Bank 0, Preset 15: Dulcimer
Bank 0, Preset 16: DrawbarOrgan
Bank 0, Preset 17: Percussive Organ
Bank 0, Preset 18: Rock Organ
Bank 0, Preset 19: Church Organ
Bank 0, Preset 20: Reed Organ
Bank 0, Preset 21: Accordian
Bank 0, Preset 22: Harmonica
Bank 0, Preset 23: Bandoneon
Bank 0, Preset 24: Nylon String Guitar
Bank 0, Preset 25: Steel String Guitar
Bank 0, Preset 26: Jazz Guitar
Bank 0, Preset 27: Clean Guitar
Bank 0, Preset 28: Palm Muted Guitar
Bank 0, Preset 29: Overdrive Guitar
Bank 0, Preset 30: Distortion Guitar
Bank 0, Preset 31

### MIDI to audio

In [4]:
from midi2audio import FluidSynth 
from pydub import AudioSegment

In [5]:
def midi_to_mp3(midi_file, mp3_file, soundfont):
    #convert MIDI to WAV using FluidSynth
    fs = FluidSynth(soundfont)
    wav_file = midi_file.replace('.midi', '.wav').replace('.mid', '.wav')
    fs.midi_to_audio(midi_file, wav_file)

    #convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav(wav_file)
    sound.export(mp3_file, format="mp3")

    print(f"Conversion complete: {mp3_file}")

In [6]:
midi_file = "output_audio_to_midi/3.mid"
mp3_file = "output_audio_to_midi/3_output.mp3"
soundfont = 'FluidR3_GM.sf2'  # Path to your SoundFont file

midi_to_mp3(midi_file, mp3_file, soundfont)

Conversion complete: output_audio_to_midi/3_output.mp3
